# Transformers Fine-tuning Step (Etapa do Fine-tuning do Transformers)
**[EN-US]**

Fine-tuning step in the DistilBERT model pre-trained on our data, to later save the model and its respective trained weights, and use it to make predictions on real-world data.

In this step we perform:
* Model definition;
* Fine-tuning the model;
* Model evaluation;
* Saving the model and trained weights.

**[PT-BR]**

Etapa do fine-tuning no modelo DistilBERT pré-treinado nos nossos dados, para depois, salvarmos o modelo e seus respectivos pesos treinados, e usarmos para realizar a predição em dados do mundo real.

Nesta etapa realizamos:
* Definição do modelo;
* Fine-tuning do modelo;
* Avaliação do modelo;
* Salvamento do modelo e dos pesos treinados.

## Table of Contents
* [Packages](#1)
* [Loading the Data](#2)
* [Transfer Learning](#3)
    * [Fine-tuning](#3.1)
* [Transformers](#4)

<a name="1"></a>
## Packages (Pacotes)
**[EN-US]**

Packages used in the system.
* [numpy](www.numpy.org): is the main package for scientific computing;
* [transformers](https://huggingface.co/docs/transformers/index): provides APIs and tools to easily download and train state-of-the-art pretrained models.

**[PT-BR]**

Pacotes utilizados no sistema.
* [numpy](www.numpy.org): é o principal pacote para computação científica;
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração.

In [74]:
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

<a name="2"></a>
## Loading the Data (Carregando os Dados)
**[EN-US]**

We will read each of the subsets from the `../data/preprocessed/` directory and plotting their dimensions.

**[PT-BR]**

Vamos ler cada um dos subsets do diretório `../data/preprocessed/` e plotando as suas dimensões.

In [31]:
files = ['train', 'valid', 'test']
datasets = []
for file in files:
    with open(f'../data/preprocessed/{file}_transformers.npy', 'rb') as f:
        datasets.append(np.load(f))

train_corpus, valid_corpus, test_corpus = datasets
print(f'Train set shape: {train_corpus.shape}\nValidation set shape: {valid_corpus.shape}\nTest set shape: {test_corpus.shape}')

Train set shape: (9682, 252)
Validation set shape: (3227, 252)
Test set shape: (3228, 252)


<a name="3"></a>
## Transfer Learning (Aprendizado por Transferência)
**[EN-US]**

One of the most powerful ideas in deep learning is that sometimes we can take the knowledge that the neural network learned in one task and apply that knowledge to a separate task. There are 3 main advantages to transfer learning:
* Reduces training time.
* Improves predictions.
* Allows us to use smaller datasets.

If we are creating a model, rather than training the weights from 0, from a random initialization, we often make much faster progress by downloading weights that someone else has already trained for days/weeks/months on the neural network architecture, use them as pre-training, and transfer them to a new, similar task that we might be interested in. This means that we can often download weights from open-source algorithms that other people took weeks or months to figure out, and then we use that as a really good initialization for our neural network.

**[PT-BR]**

Uma das ideias mais poderosas do deep learning, que às vezes, podemos pegar o conhecimento que a rede neural aprendeu em uma tarefa e aplicar esse conhecimento em uma tarefa separada. Há 3 vantagens principais no transfer learning:
* Reduz o tempo de treinamento.
* Melhora as previsões.
* Nos permite usar datasets menores.

Se estamos criando um modelo, em vez de treinarmos os pesos do 0, a partir de uma inicialização aleatória, geralmente progredimos muito mais rápido baixando pesos que outra pessoa já treinou por dias/semanas/meses na arquitetura da rede neural, as usamos como pré-treinamento, e as transferimos para uma nova tarefa semelhante na qual possamos estar interessados. Isso significa que muitas vezes podemos baixar pesos de algoritmos open-source, que outras pessoas levaram semanas ou meses para descobrir, e então, usamos isso como uma inicialização muito boa para nossa rede neural.

<a name="3.1"></a>
### Fine-tuning
**[EN-US]**

We take the weights from an existing pre-trained model using transfer learning and then tweak them a bit to ensure they work for the specific task we are working on. Let's say we pre-trained a model that predicts movie evaluation, and now we're going to create a model to evaluate courses. One way to do this is, by locking all the weights that we already have pre-trained, and then we add a new output layer, or perhaps, a new feed forward layer and an output layer that will be trained, while we keep the rest locked and then we only train our new network, the new layers that we just added. We can slowly unfreeze the layers, one at a time.

Many of the low-level features that the pre-trained model learned from a very large corpus, like the structure of the text, the nature of the text, this can help our algorithm do better in the sentiment classification task and faster or with less data, because maybe the model has learned enough what the structures of different texts are like and some of that knowledge will be useful. After deleting the output layer of a pre-trained model, we don't necessarily need to create just the output layer, but we can create several new layers.

We need to remove the output layer from the pre-trained model and add ours, because the neural network can have a softmax output layer that generates one of 1000 possible labels. So we remove this output layer and create our own output layer, in this case a sigmoid activation.

* With a small training set, we think of the rest of the layers as `frozen`, so we freeze the parameters of these layers, and only train the parameters associated with our output layer. This way we will obtain very good performance, even with a small training set.

* With a larger training set, we can freeze fewer layers and then train the layers that were not frozen and our new output layer. We can use the layers that are not frozen as initialization and use gradient descent from them, or we can also eliminate these layers that are not frozen and use our own new hidden layers and our own output layer. Any of these methods could be worth trying.

* With a much larger training set, we use this pre-trained neural network and its weights as initialization and train the entire neural network, just changing the output layer, with labels that we care about.

**[PT-BR]**

Pegamos os pesos de um modelo pré-treinado existente, utilizando transfer learning e, em seguida, ajustamos um pouco para garantir que funcionem na tarefa específica em que estamos trabalhando. Digamos que pré-treinamos um modelos que prevê a avaliação de filmes, e agora vamos criar um modelo para avaliar cursos. Uma maneira de fazer isso é, bloqueando todos os pesos que já temos pré-treinados e, em seguida, adicionamos uma nova output layer, ou talvez, uma nova feed forward layer e uma output layer que serão treinadas, enquanto mantemos o restante bloqueado e, em seguida, treinamos apenas a nossa nova rede, as novas layers que acabamos de adicionar. Podemos descongelar lentamente as layers, uma de cada vez.

Muitas das features de baixo nível que o modelo pré-treinado aprendeu a partir de um corpus muito grande, como a estrutura do texto, a natureza do texto, isso pode ajudar nosso algoritmo a se sair melhor na tarefa de classificação de sentimentos e mais rápido ou com menos dados, porque talvez o modelo tenha aprendido o suficiente como são as estruturas de textos diferentes e parte desse conhecimento será útil. Após excluirmos a output layer de um modelo pré-treinado, não precisamos necessariamente criar apenas a output layer, mas podemos criar várias novas layers.

Precisamos remover a output layer do modelo pré-treinado e adicionar a nossa, porque a rede neural pode ter uma softmax output layer que gera um dos 1000 labels possíveis. Então removemos essa output layer e criamos a nossa própria output layer, nesse caso uma ativação sigmoid.

* Com um training set pequeno, pensamos no restante das layers como `congeladas`, então congelamos os parâmetros dessas layers, e treinamos apenas os parâmetros associados à nossa output layer. Dessa forma obteremos um desempenho muito bom, mesmo com um training set pequeno.

* Com um training set maior, nós podemos congelar menos layers e então treinar as layers que não foram congeladas e a nossa nova output layer. Podemos usar as layers que não estão congeladas como inicialização e usar o gradient descent a partir delas, ou também podemos eliminar essas layers que não estão congeladas e usamos nossas próprias hidden layers novas e nossa própria output layer. Qualquer um desses métodos pode valer a pena tentar.

* Com um training set muito maior usamos essa rede neural pré-treinada e os seus pesos como inicialização e treinamos toda a rede neural, apenas alterando a output layer, com labels que nos importamos.

<a name="4"></a>
## Transformers
**[EN-US]**

<img align='center' src='../figures/transformers.png' style='width:400px;'>
Transformer is a purely attention-based model that was developed by Google to solve some problems with RNNs, it is difficult to fully exploit the advantages of parallel computing, due to its problems arising from its sequential structure. In a seq2seq RNN, we need to go through each word of our input, sequentially by the encoder, and it is done in a similar sequential way by the decoder, without parallel computation. For this reason, there is not much room for parallel calculations. The more words we have in the input sequence, the more time it will take to process it.

With large sequences, that is, with many $T$ sequential steps, information tends to get lost in the network (loss of information) and problems of vanishing gradients arise related to the length of our input sequences. LSTMs and GRUs help a little with these problems, but even these architectures stop working well when they try to process very long sequences due to the `information bottleneck`.
* `Loss of information`: it is more difficult to know whether the subject is singular or plural as we move away from the subject.
* `Vanishing gradients`: When we calculate backprop, the gradients can become very small and as a result the model will not learn anything.

Transformers are based on attention and do not require any sequential calculation per layer, requiring only a single step. Furthermore, the gradient steps that need to be taken from the last output to the first output in a transformer are just 1. For RNNs, the number of steps increases with longer sequences. Finally, transformers do not suffer from problems of vanishing gradients related to the length of the sequences.

Transformers do not use RNNs, attention is all we need, and only a few linear and non-linear transformations are usually included. The transformers model was introduced in 2017 by Google researchers, and since then, the transformer architecture has become standard for LLMs. Transformers have revolutionized the field of NLP.

The transformer model uses `scaled dot-product attention`. The first form of attention is very efficient in terms of computation and memory, because it consists only of matrix multiplication operations. This engine is the core of the model and allows the transfomer to grow and become more complex, while being faster and using less memory than other comparable model architectures.

In the transformer model, we will use the `Multi-Head Attention layer`, this layer is executed in parallel and has several $h$ scaled dot-product attention mechanisms and several linear transformations of input queries, keys and values. In this layer, linear transformations are trainable parameters.
$$\text{ Attention}(Q, K, V) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} \right) V$$
<img align='center' src='../figures/attention.png' style='width:600px;'>

`Encoders` transformers start with a multi-head attention module that performs `self-attention` on the input sequence. This is followed by a residual connection and normalization, then a feed forward layer and another residual connection and normalization. The encoder layer is repeated $N_x$ times.
* Self-attention: each word in the input corresponds to each other word in the input sequence
* Thanks to the self-attention layer, the encoder will provide a contextual representation of each of our inputs.


The `decoder` is built in a similar way to the encoder, with multi-head attention, residual connections and normalization modules. The first attention module is masked (`Masked Self-Attention`) so that each position only serves the previous positions, it blocks the flow of information to the left. The second attention module (`Encoder-Decoder Attention`) takes the encoder output and allows the decoder to attend to all items. This entire decoder layer is also repeated several $N_x$ times, one after the other.
$$\text{ Masked Self-Attention } = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + M \right) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + \text{ mask matrix } \begin{pmatrix} 0 & -\infty & -\infty \\ 0 & 0 & -\infty \\ 0 & 0 & 0 \end{pmatrix} \right)$$

Transformers also incorporate a `positional encoding stage` ($PE$), which encodes the position of each input in the sequence, that is, the sequential information. This is necessary because transformers don't use RNNs, but word order is relevant to any language. Positional encoding is trainable, just like word embeddings.
$$\begin{align*}
& \text{PE}_{(\text{pos, }2i)} = \text{sin}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right) \\
& \text{PE}_{(\text{pos, }2i + 1)} = \text{cos}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right)
\end{align*}$$

First, the embedding on the input is calculated and the positional encodings are applied. So, this goes to the encoder which consists of several layers of Multi-Head Attention modules, then the decoder receives the output sequence shifted one step to the right and the encoder outputs. The decoder output is transformed into output probabilities using a linear layer with a softmax activation. This architecture is easy to parallelize compared to RNNs models and can be trained much more efficiently on multiple GPUs. It can also be scaled to learn multiple tasks on increasingly larger datasets. Transformers are a great alternative to RNNs and help overcome these problems in NLP and many fields that process sequential data.

We will fine-tune the DistilBERT model, which is a small, fast, cheap and lightweight Transformer model, trained by distilling the base BERT model. It has 40% fewer parameters than bert-base-uncased, runs 60% faster, and preserves more than 95% of BERT's performance as measured in the GLUE (General Language Understanding Evaluation) benchmark.

[Hugging Face](https://huggingface.co/) (🤗) is the best resource for pre-trained transformers. Its open source libraries make it simple to download, fine-tune, and use transformer models like DeepSeek, BERT, Llama, T5, Qwen, GPT-2, and more. And the best part, we can use them together with TensorFlow, PyTorch or Flax. In this notebook, I use 🤗 transformers to use the `DistilBERT` model for sentiment classification. For the pre-processing step, we use the tokenizer (in the notebook `03_preprocessing.ipynb`), and the DistilBERT fine-tuning `distilbert-base-uncased-finetuned-sst-2-english` pre-trained in the code below. To do this, we initialize the DistilBertForSequenceClassification class and define the desired pre-trained model.

Defining the pre-trained model that we will do the fine-tuning.

**[PT-BR]**

<img align='center' src='../figures/transformers.png' style='width:400px;'>
Transformer é um modelo puramente baseado em attention que foi desenvolvido pelo Google para solucionar alguns problemas com RNNs, é difícil de explorar totalmente as vantagens da computação paralela, devido aos seus problemas decorrentes de sua estrutura sequencial. Em um RNN seq2seq, precisamos passar por cada palavra de nosso input, de forma sequencial pelo encoder, e é feito de uma forma sequencial similar pelo decoder, sem computação paralela. Por esse motivo, não há muito espaço para cálculos paralelos. Quanto mais palavras temos na sequência de input, mais tempo será necessário para processá-la.

Com sequências grandes, ou seja, com muitos $T$ steps sequenciais, as informações tendem a se perder na rede (loss of information) e problemas de vanishing gradients surgem relacionados ao comprimento de nossas input sequences. LSTMs e GRUs ajudam um pouco com esses problemas, mas mesmo essas arquitutras param de funcionar bem quando tentam processar sequências muito longas devido ao `information bottleneck`.
* `Loss of information`: é mais difícil saber se o sujeito é singular ou plural à medida que nos afastamos do sujeito.
* `Vanishing gradients`: quando calculamos o backprop, os gradients podem se tornar muito pequenos e, como resultado, o modelo não aprenderá nada.

Os transformers se baseiam em attention e não exigem nenhum cálculo sequencial por layer, sendo necessário apenas um único step. Além disso, os steps de gradient que precisam ser realizados do último output para o primeiro output em um transformer são apenas 1. Para RNNs, o número de steps aumenta com sequências mais longas. Por fim, os transformers não sofrem com problemas de vanishing gradients relacionados ao comprimento das sequências.

Transformers não usam RNNs, attention é tudo o que precisamos, e apenas algumas transformações lineares e não lineares geralmente são incluídas. O modelo transformers foi introduzido em 2017 por pesquisadores do Google, e desde então, a arquitetura do transformer se tornou padrão para LLMs. Os transformers revolucionaram o campo de NLP.

O modelo transformer usa `scaled dot-product attention`. A primeira forma da attention é muito eficiente em termos de computação e memória, porque consiste apenas em operações de multilpicações de matrizes. Esse mecanismo é o núcleo do modelo e permite que o transfomer cresça e se torne mais complexo, sendo mais rápido e usando menos memória do que outras arquiteturas de modelos comparáveis.

No modelo transformer, usaremos a `Multi-Head Attention layer`, essa layer é executada em paralelo e tem vários mecanismos $h$ scaled dot-product attention e várias transformações lineares dos input queries, keys e values. Nessa layer, as transformações lineares são parâmetros treináveis.
$$\text{ Attention}(Q, K, V) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} \right) V$$
<img align='center' src='../figures/attention.png' style='width:600px;'>

Os transformers `encoders` começam com um módulo multi-head attention que executa a `self-attention` na input sequence. Isso é seguido por uma residual connection e normalização, em seguida, uma feed forward layer e outra residual connection e normalização. A encoder layer é repetida $N_x$ vezes.
* Self-attention: cada palavra no input corresponde a cada outra palavra no input sequence
* Graças à self-attention layer, o encoder fornecerá uma representação contextual de cada um de nossos inputs.


O `decoder` é construído de forma similar ao encoder, com módulos multi-head attention, residual connections e normalização. O primeiro módulo de attention é mascarado (`Masked Self-Attention`) de forma que cada posição atenda apenas às posições anteriores, ele bloqueia o fluxo de informações para a esquerda. O segundo módulo de attention (`Encoder-Decoder Attention`) pega o output do encoder e permite que o decoder atenda a todos os itens. Toda essa decoder layer também é repetida várias $N_x$ vezes, uma após a outra.
$$\text{ Masked Self-Attention } = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + M \right) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + \text{ mask matrix } \begin{pmatrix} 0 & -\infty & -\infty \\ 0 & 0 & -\infty \\ 0 & 0 & 0 \end{pmatrix} \right)$$

Os transformers também incorporam um `positional encoding stage` ($PE$), que codifica a posição de cada input na sequência, ou seja, as informações sequenciais. Isso é necessário porque os transformers não usam RNNs, mas a ordem das palavras é relevante para qualquer idioma. A positional encoding é treinável, assim como as word embeddings.
$$\begin{align*}
& \text{PE}_{(\text{pos, }2i)} = \text{sin}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right) \\
& \text{PE}_{(\text{pos, }2i + 1)} = \text{cos}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right)
\end{align*}$$

Primeiro, é calculado o embedding sobre o input e as positional encodings são aplicadas. Então, isso vai para o encoder que consiste em várias layers de módulos de Multi-Head Attention, em seguida, o decoder recebe a sequência de output deslocada um step para a direita e os outputs do encoder. O output do decoder é transformado em probabilidades de outputs usando uma layer linear com uma ativação softmax. Essa arquitetura é fácil de paralelizar em comparação com os modelos RNNs e pode ser treinada com muito mais eficiência em várias GPUs. Ela também pode ser escalada para aprender várias tarefas em datasets cada vez maiores. Transformers são uma ótima alternativa aos RNNs e ajudam a superar esses problemas em NLP e em muitos campos que processam dados sequenciais.

Faremos o fine-tuning no modelo DistilBERT, que é um modelo Transformer pequeno, rápido, barato e leve, treinado pela destilação do modelo base BERT. Ele tem 40% menos parâmetros que o bert-base-uncased, roda 60% mais rápido e preserva mais de 95% do desempenho do BERT conforme medido no benchmark GLUE (General Language Understanding Evaluation).

[Hugging Face](https://huggingface.co/) (🤗) é o melhor recurso para transformers pré-treinados. Suas bibliotecas de código aberto simplificam o download, o fine-tuning e o uso de modelos de transformers como DeepSeek, BERT, Llama, T5, Qwen, GPT-2 e muito mais. E a melhor parte, podemos usá-los junto com TensorFlow, PyTorch ou Flax. Neste notebook, utilizo transformers 🤗 para usar o modelo `DistilBERT` para classificação de sentimento. Para a etapa do pré-processamento, usamos o tokenizador (no notebook `03_preprocessing.ipynb`), e no fine-tuning do DistilBERT `distilbert-base-uncased-finetuned-sst-2-english` pré-treinado no código abaixo. Para isso inicializamos a classe DistilBertForSequenceClassification e definidos o modelo pré-treinado desejado.

Definindo o modelo pré-treinado que faremos o fine-tuning.

In [36]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

**[EN-US]**

Setting the hyperparameters, using the Hugging Face Trainer object to fine-tune the model.
* The pre-trained model with fine-tuning is already being saved in the `../models/` directory, defined when defining the hyperparameters.

**[PT-BR]**

Definindo os hiperparâmetros, usando o objeto Trainer do Hugging Face para realizar o fine-tuning do modelo.
* O modelo pré-treinado com o fine-tuning já está sendo salvo no diretório `../models/`, definido na definição dos hiperparâmetros.

In [ ]:
# Fine-tuning hyperparameters
# Hiperparâmetros do fine-tuning
training_args = TrainingArguments(
    output_dir='../models/transformers_results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=20,
    weight_decay=0.01,
    logging_steps=50
)

# Trainer object
# Objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_corpus,
    eval_dataset=valid_corpus,
)
trainer.train()

**[EN-US]**

Evaluating model performance with fine-tuning in the test set.

**[PT-BR]**

Avaliando o desempenho do modelo com fine-tuning no test set.

In [ ]:
print(f'Test set evaluate: {trainer.evaluate(test_corpus)}')